In [1]:
from __future__ import division
import numpy as np
import pandas as pd
import StringIO
from datetime import date, timedelta as td
import xlsxwriter
import os
import json
import urllib2
import base64
import csv

In [2]:
import argparse

from apiclient.discovery import build
import httplib2
from oauth2client import client
from oauth2client import file
from oauth2client import tools

In [3]:
def get_service(api_name, api_version, scope, client_secrets_path):
  """Get a service that communicates to a Google API.

  Args:
    api_name: string The name of the api to connect to.
    api_version: string The api version to connect to.
    scope: A list of strings representing the auth scopes to authorize for the
      connection.
    client_secrets_path: string A path to a valid client secrets file.

  Returns:
    A service that is connected to the specified API.
  """
  # Parse command-line arguments.
  parser = argparse.ArgumentParser(
      formatter_class=argparse.RawDescriptionHelpFormatter,
      parents=[tools.argparser])
  flags = parser.parse_args([])

  # Set up a Flow object to be used if we need to authenticate.
  flow = client.flow_from_clientsecrets(
      client_secrets_path, scope=scope,
      message=tools.message_if_missing(client_secrets_path))

  # Prepare credentials, and authorize HTTP object with them.
  # If the credentials don't exist or are invalid run through the native client
  # flow. The Storage object will ensure that if successful the good
  # credentials will get written back to a file.
  storage = file.Storage(api_name + '.dat')
  credentials = storage.get()
  if credentials is None or credentials.invalid:
    credentials = tools.run_flow(flow, storage, flags)
  http = credentials.authorize(http=httplib2.Http())

  # Build the service object.
  service = build(api_name, api_version, http=http)

  return service

In [4]:
scope = ['https://www.googleapis.com/auth/analytics.readonly']
service = get_service('analytics', 'v3', scope, 'client_secrets.json')

In [7]:
def get_ga_data(service, config_dict):
    
    df = pd.DataFrame()
    
    for i in pd.date_range(start_date, end_date):
        
        out = service.data().ga().get(
            ids='ga:' + '86300562',
            start_date=str(i.date()),
            end_date=str(i.date()),
            max_results=10000,
            metrics=','.join(config_dict['metric_list']),
            dimensions=','.join(config_dict['dimension_list']),
            filters=';'.join(config_dict['filter_list'])).execute()

        col_list = []

        for i in np.arange(0,len(out['columnHeaders'])):
            line = re.sub('ga.', '', out['columnHeaders'][i][u'name'])
            col_list.append(line)
        try:
            df = df.append(pd.DataFrame(out['rows'],columns=col_list))
        except:
            len(out) == 0
            
    return df

In [31]:
transactions_dict = {
    'filter_list': [
'ga:campaign==mitx',

],
    'dimension_list': [
'ga:date',
'ga:adContent',
],
    'metric_list': [
'ga:sessions',
'ga:transactions',
'ga:totalValue'
],
}

In [32]:
enrollments_dict = {
    'filter_list': [
'ga:eventAction==edx.course.enrollment.activated',
'ga:campaign==mitx',

],
    'dimension_list': [
'ga:date',
'ga:adContent'
],
    'metric_list': [
'ga:uniqueEvents'
],
}

In [33]:
start_date = '2016-09-01'
end_date = '2016-12-12'

In [34]:
enrollments_df = get_ga_data(service, enrollments_dict)
transactions_df = get_ga_data(service, transactions_dict)

In [39]:
master_df = enrollments_df.merge(transactions_df, how = 'outer', on = ['date','adContent'])

In [47]:
master_df.to_csv('./../script_output/mit_marketing_perf_20160901_20161212.csv', encoding='utf-8')